# Match most similar competency

## Existing competencies

In [53]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [54]:
competencys = ['Creativity', 'Employee loyalty', 'Reliability', 'Leadership', 'Passion', 'People Skills', 'Proactivity', 'Team Work']
for i in range(8):
    print "{}:{}".format(i, competencys[i])


0:Creativity
1:Employee loyalty
2:Reliability
3:Leadership
4:Passion
5:People Skills
6:Proactivity
7:Team Work


In [55]:
descriptions = ['Ability to think in a divergent way, and generate novel ideas.',
                'Be committed to the success of the organization and believe that working for this organization is their best option.',
                'Ability to performing to perform when needed, finishing projects and meeting deadlines.', 
                'Ability to engage and facilitate others to create a better result.', 
                'Desire to provide long-term commitment to their organization, demonstrate peak performance, and maintain increased tenure with the organization. With the intense desire or enthusiasm for the work.', 
                'Ability to establish social connections and understand the mindset of others.', 
                'Take the initiative and responsibility in improving business, rather than looking for causes in outside circumstances or other people.',
                'Ability to adapt to the needs of a group of people, while supporting the work of others.']

## load dictionary and corpus

In [56]:
from gensim import corpora, similarities, models

dictionary = corpora.Dictionary.load_from_text('/mnt/local/var/seedlink/data/dictionary/dict_en.txt')
print(dictionary)

Dictionary(50920 unique tokens: [u'sucess', u"cake'", u'mid-week', u'stallions', u'sonja']...)


In [57]:
import codecs
import json
path = '/mnt/local/var/seedlink/data/nltk_data/corpora/prepcorp/en_cc.json'
with codecs.open(path, 'r', 'utf8') as fp:
    jdata = fp.read()
data = json.loads(jdata)
corpus = data

In [58]:
for i in range(10):
    words =  [dictionary[t[0]] for t in corpus[i]]
    print corpus[i]
    print ' '.join(words)

[[3, 1], [79, 1], [142, 1], [177, 1], [2675, 1], [2912, 1], [5182, 1]]
a is like big kevin kid smith
[[3, 1], [29, 1], [33, 1], [57, 1], [79, 1], [1160, 1], [1327, 1], [1436, 1], [13435, 1]]
a his of that is juvenile humor s sophisticated
[[3, 1], [4, 1], [174, 1], [282, 1], [325, 1], [357, 1], [359, 1], [575, 1], [706, 1], [756, 1], [1171, 1], [1344, 1], [4875, 1], [5929, 1], [39194, 1]]
a and up he also books comic star having few written wars loves grew idolizing
[[3, 1], [4, 1], [26, 1], [33, 1], [282, 1], [295, 1], [325, 1], [831, 1], [1436, 1], [2030, 1], [2169, 1], [4348, 1], [5182, 1], [6362, 1], [6552, 1], [8210, 1], [8365, 1], [9187, 1], [20250, 1], [22120, 1]]
a and has of he own also teenagers s mostly following fellow smith adolescent composed students college cult minded grownups
[[4, 2], [14, 1], [21, 1], [30, 2], [57, 1], [79, 2], [175, 2], [277, 1], [282, 2], [502, 1], [979, 1], [1093, 1], [1811, 1], [3874, 1], [4990, 1], [5182, 1], [6672, 1], [39196, 1]]
and to but in

## New competency

In [140]:
new_comp = """
Charisma	Able to establish social connections and a broad network
Confidence	Acts on his/her own initiative with confidence
Creativity	Seeks change and performs creatively
Entrepreneurial	Keeps aware of both organizational issues and market opportunities
Goal Orientated	Works systematically and drives the project to its goals
Goal Orientation	Work systematically and drive the project to the target goals
Growth Potential	Growth Potential
Integrity	Acts with integrity and shows social responsibility
Intellect	Flexible application of knowledge to differing contexts
Job Performance	Job Performance
Leadership	Identifies talents, empowers and motivates other
Logic	Logical thinking and rational analysis
Organisation	Sets objectives, balances resources and time, monitors progress
Organization	Sets objectives, balances resources and time, monitors progress
Performance	Job Performance
Proactive	Proactively deals with ambiguity
Process driven	Demonstrate commitment, accountablity, follow policies and procedures of the organizations
Resilient	Coping and resilient under stressful environment
Self motivated	Work enthusiastically for the achievement of self-development
Strategic Competency	Strategic Competency
Strategy	Foresight for potential problems and comes up with appropriate solutions
Teamwork	Adapts to the team; Supports others
"""

lines = new_comp.lower().strip().split('\n')
new_comp_dict = {splitted[0]:splitted[1] for splitted in [line.split('\t') for line in lines]}

## Tfidf

In [59]:
tfidf = models.TfidfModel(corpus)

In [60]:
desc_bow = [dictionary.doc2bow(desc.lower().split()) for desc in descriptions]
desc_tfidf = tfidf[desc_bow]

## LSI

In [70]:
lsi = models.LsiModel(tfidf[corpus], id2word=dictionary, num_topics=300)

In [71]:
lsi.add_documents(desc_tfidf)

In [72]:
index = similarities.MatrixSimilarity(lsi[desc_tfidf]) # transform corpus to LSI space and index it

0:Creativity

1:Employee loyalty

2:Reliability

3:Leadership

4:Passion

5:People Skills

6:Proactivity

7:Team Work

In [141]:
# query by competency name
for comp in new_comp_dict:
    query = comp.lower()
    query_bow = dictionary.doc2bow(query.split())
    query_lsi = lsi[tfidf[query_bow]]

    sims = index[query_lsi]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    print "{} -> {} {}".format(comp, competencys[sims[0][0]], sims[0][1])

strategic competency -> People Skills 0.21805781126
growth potential -> Passion 0.0826770737767
integrity -> Passion 0.130976200104
intellect -> Reliability 0.103481665254
confidence -> Passion 0.143962875009
goal orientated -> Employee loyalty 0.217423051596
job performance -> Employee loyalty 0.0442851819098
organisation -> Employee loyalty 0.0364584811032
creativity -> Leadership 0.0557141378522
self motivated -> People Skills 0.0790649056435
strategy -> Team Work 0.167022421956
charisma -> People Skills 0.134312421083
teamwork -> People Skills 0.0189893096685
resilient -> Team Work 0.0666694194078
performance -> Employee loyalty 0.0230101142079
goal orientation -> Employee loyalty 0.209044247866
leadership -> Reliability 0.168288677931
entrepreneurial -> People Skills 0.0364505276084
logic -> Team Work 0.0638348832726
organization -> Employee loyalty 0.349326014519
process driven -> Proactivity 0.115234076977
proactive -> Creativity 0.0


In [138]:
# query by competency desc
for comp in new_comp_dict:
    query = new_comp_dict[comp].lower()
    query_bow = dictionary.doc2bow(query.split())
    query_lsi = lsi[tfidf[query_bow]]

    sims = index[query_lsi]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    print "{} -> {} {}".format(comp, competencys[sims[0][0]], sims[0][1])

strategic competency -> People Skills 0.21805781126
growth potential -> Passion 0.0826770737767
integrity -> Passion 0.424755781889
intellect -> People Skills 0.728225588799
confidence -> Passion 0.230781331658
goal orientated -> People Skills 0.224078446627
job performance -> Employee loyalty 0.0442851819098
organisation -> People Skills 0.316874146461
creativity -> People Skills 0.291215986013
self motivated -> Team Work 0.5838149786
strategy -> Passion 0.306700468063
charisma -> People Skills 0.587397158146
teamwork -> People Skills 0.504117012024
resilient -> People Skills 0.355710297823
performance -> Employee loyalty 0.0442851819098
goal orientation -> Team Work 0.623666167259
leadership -> Proactivity 0.389719635248
entrepreneurial -> People Skills 0.223295211792
logic -> People Skills 0.294291198254
organization -> People Skills 0.316874146461
process driven -> People Skills 0.646625757217
proactive -> Passion 0.556239128113
